In [1]:
#|default_exp 03.02_app

## Revisiting Creating Simple Neural Networks

As I progress through Part 2 of the fastai course, I've discovered the value of occasionally retracing my steps to assess my grasp on the fundamental concepts. In my journey through Part 1, I recall Chapter 4 of the book being particularly daunting, as it covered a broad range of essential topics early on. However, this comprehensive approach was necessary to establish a strong foundation for everything I would subsequently learn.

Whenever I revisit those earlier chapters and analyze the code line by line, I'm often pleasantly surprised by the level of intuition I've developed in certain areas. Simultaneously, I'm also confronted with aspects that still elude my immediate understanding. Engaging in quick drills to reinforce these concepts has proven to be an effective method for staying sharp and achieving complete comprehension, even if it may feel somewhat repetitive. It's like eating vegetables as a child – you may not want to, but it's ultimately beneficial and necessary for growth. I believe that through a similar positive feedback loop, I will acquire a taste for this process and derive great rewards from it.

So, grab a fork and let's dig in

## Predicting Bank Loan Default from Synthetic Data

For this exercise I'm going to create synthetic data to predict whether a bank loan will default. I've chosen the following to create as features:

- Loan Amount 
- Term 
- Interest Rate 
- Borrower's Income 
- Borrower's Credit Score

## Generate Synthetic Data

In [2]:
import torch

if torch.cuda.is_available():
    torch.cuda.set_device(0)
    torch_device = torch.device("cuda")
else:
    torch_device = torch.device("cpu")


We're first going to create the data for 1000 samples randomly with `torch.normal`, to which we'll pass arguments for a *mean*, *standard deviation*, and *size*.

The size could be either a tuple or a list, but for this exercise we'll just use a tuple with a single value.

Also worth noting that `term` will be using `randint` rather than `normal`, because loan terms are generally in whole months.

In [3]:
#|export
n_samples = 1000
loan_amount = torch.normal(5000., 1500, size=(n_samples,))  # average loan amount is $5000
term = torch.randint(12, 60, size=(n_samples,))  # loan term varies between 1 and 5 years
interest_rate = torch.normal(0.05, 0.01, size=(n_samples,))  # average interest rate is 5%
income = torch.normal(50000, 10000, size=(n_samples,))  # average income is $50,000
credit_score = torch.normal(600, 50, size=(n_samples,))  # average credit score is 600

Next we'll stack the tensors and create a target variable where 10% of loans default, and convert them to *float32* data types in the process

In [4]:
#|export
x = torch.column_stack([loan_amount, term, interest_rate, income, credit_score]).float()
y = torch.distributions.categorical.Categorical(torch.tensor([0.9, 0.1])).sample((n_samples,)).float()

And normalize the data

In [5]:
x = torch.nn.functional.normalize(x)

We'll also need to add a unit axis to our dependent variable so that it can be properly broadcast.

In [6]:
y.shape

torch.Size([1000])

This can be done using either `view` or specifying `None` on the axis we want to create

In [7]:
y.view(-1,1).shape

torch.Size([1000, 1])

In [8]:
y[:, None].shape

torch.Size([1000, 1])

In [9]:
#|export
y = y[:, None];

In [10]:
y.shape

torch.Size([1000, 1])

Now that we have our data, let's move onto the creation of our neural network!

## Creating a Simple Neural Network with Pytorch

For this exercise, we're going to use 2 linear transformation layers, a *ReLU* activation layer with 3 nodes, and a sigmoid activation for the output to get our probability, between 0 and 1. To add a bit of an extra challege for myself I'm going to combine the steps into a class. Classes are something that I haven't had to use much yet, and this seems like a great opportunity to give myself some progressive overload on the mental muscles.

In [ ]:
#|export
import torch.nn.functional as F

class SimpleNet(torch.nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.lin_1 = torch.nn.Linear(5, 3)
        self.lin_2 = torch.nn.Linear(3, 1)

    def forward(self, x):
        x = self.lin_1(x)
        x = F.relu(x)
        x = self.lin_2(x)
        output = torch.sigmoid(x)
        return output

If the GPU is available, we're going to want to use it, so let's initialize our `SimpleNet` and make sure it's running on cuda. 

We'll do the same for `x` and `y` as well

In [ ]:
#|export
model = SimpleNet().to(torch_device)
x = x.to(torch_device)
y = y.to(torch_device)

In [ ]:
x.device

Now we'll need to define a *Loss Function* to evaluate our model with, for simplicity we'll use *Mean Squared Error(MSE)*. Very wrong, I know, but just for this exercise.

In [ ]:
#|export
def mse(output, target):
    return (output - target).pow(2).mean()

We'll also need to specify our learning rate, and how we want to optimize our model, which is *Stochastic Gradient Descent(SGD)* in this case

In [ ]:
#|export
lr = 1e-2

In [ ]:
#|export
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [ ]:
print(model.parameters)

Now it's time to put it all in a loop, and run it a few times

In the loop we're going to:
- Generate predictions with our model
- Calculate the loss with our loss function
- Compute the gradients with respect to our parameters
- Update the parameters using our optimizer
- Zero the gradients to prepare for the next iteration

In [ ]:
model(x)

In [ ]:
#|export
from tqdm import tqdm

epochs = 100
# loss_fn = torch.nn.BCELoss()

for epoch in tqdm(range(epochs)):
    # compute predictions
    preds = model(x)
    # compute loss
    loss = mse(preds, y)
    
    print(f"Epoch: {epoch}, Loss: {loss}")
    # backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    

In [ ]:
import nbdev
nbdev.export.nb_export("03.02_gpt_teach_nn.ipynb", "03.02_app")